In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
plt.rcParams['figure.figsize']= (15,10)

dengue_train = pd.read_csv('./data/dengue_features_train.csv')
dengue_test = pd.read_csv('./data/dengue_features_test.csv')
dengue_labels_train = pd.read_csv('./data/dengue_labels_train.csv')

print("Dengue Features Train shape:", dengue_train.shape)
print("Dengue Features Test shape:", dengue_test.shape)
print("Dengue Labels Train shape:", dengue_labels_train.shape)

Dengue Features Train shape: (1456, 24)
Dengue Features Test shape: (416, 24)
Dengue Labels Train shape: (1456, 4)


## Dengue Features Dataset
This describes the factors of climate and vegetation during the training period.
- city: City abbreviations: sj for San Juan and iq for Iquitos
- week_start_date: Date given in yyyy-mm-dd format
- weekofyear: The number of week of the year
- year: The year in yyyy format

- station_max_temp_c: Maximum temperature
- station_min_temp_c: Minimum temperature
- station_avg_temp_c: Average temperature
- station_precip_mm: Total precipitation
- station_diur_temp_rng_c: Diurnal temperature range

- precipitation_amt_mm: Total precipitation

- reanalysis_sat_precip_amt_mm: Total precipitation
- reanalysis_dew_point_temp_k: Mean dew point temperature
- reanalysis_air_temp_k: Mean air temperature
- reanalysis_relative_humidity_percent: Mean relative humidity
- reanalysis_specific_humidity_g_per_kg: Mean specific humidity
- reanalysis_precip_amt_kg_per_m2: Total precipitation
- reanalysis_max_air_temp_k: Maximum air temperature
- reanalysis_min_air_temp_k: Minimum air temperature
- reanalysis_avg_temp_k: Average air temperature
- reanalysis_tdtr_k: Diurnal temperature range

- nvdi_se: Pixel southeast of city centroid
- nvdi_sw: Pixel southwest of city centroid
- nvdi_ne: Pixel northeast of city centroid
- nvdi_nw: Pixel northwest of city centroid

## Dengue Labels Dataset
This describes the factors of climate and vegetation during the training period.
- city: City abbreviations: sj for San Juan and iq for Iquitos
- year: The year in yyyy format
- weekofyear: The number of week of the year
- total_cases: The total number of cases active in that city for the week and year.